In [ ]:
import pandas as pd

csv_path = '/Users/theoverdelhan/Documents/TRADING/MYR/global_competitor_analysis/gap_study/TSLA_5d_quotes.csv'
df_price = pd.read_csv(csv_path)

df_price['ask_price'] = df_price['ap']
df_price['bid_price'] = df_price['bp']

df_price['ask_size'] = df_price['as']
df_price['bid_size'] = df_price['bs']

df_price['mid_price'] = (df_price['ask_price'] + df_price['bid_price']) / 2

df_price['spread_ask_bid'] = ((df_price['ask_price'] - df_price['bid_price']) /df_price['ask_price']) * 100

# df_price = df_price[900000:1800000]

df_price['time'] = pd.to_datetime(df_price['t'], unit='ns', utc=True)

df_price['time'] = df_price['time'].dt.round('ms')
df_price['time'] = df_price['time'].astype('int64') // 10**6


columns_to_drop = ['as', 'bs', 'ap', 'bp', 't', 'c', 'z']
df_price.drop(columns=columns_to_drop, inplace=True)

df_price

,ax,bx,ask_price,bid_price,ask_size,bid_size,mid_price,spread_ask_bid,time
0,Z,K,261.24,261.09,2,2,261.165,0.057418,1743596492118
1,Z,Q,261.24,261.11,2,1,261.175,0.049763,1743596492203
2,Z,Q,261.24,261.12,2,1,261.180,0.045935,1743596492490
3,Z,K,261.24,261.09,2,2,261.165,0.057418,1743596493083
4,Z,K,261.24,261.10,2,2,261.170,0.053591,1743596493106
...,...,...,...,...,...,...,...,...,...
3387783,K,Q,222.88,222.80,2,7,222.840,0.035894,1744028489882
3387784,K,Q,222.88,222.80,2,4,222.840,0.035894,1744028491030
3387785,K,Q,222.88,222.80,2,3,222.840,0.035894,1744028491030
3387786,K,Q,222.88,222.80,2,2,222.840,0.035894,1744028491030


In [ ]:
import pandas as pd

sorted_spreads = df_price['spread_ask_bid'].sort_values(ascending=False)
top_30_percent = sorted_spreads.head(int(len(df_price) * 0.1))
mean_top_spreads = top_30_percent.mean()
print(mean_top_spreads)

0.1447946993488734


In [ ]:
import pandas as pd

df = df_price
q1 = df['spread_ask_bid'].quantile(0.25)
q2 = df['spread_ask_bid'].quantile(0.50)
q3 = df['spread_ask_bid'].quantile(0.75)


quartile_1 = df[df['spread_ask_bid'] <= q1]['spread_ask_bid']
quartile_2 = df[(df['spread_ask_bid'] > q1) & (df['spread_ask_bid'] <= q2)]['spread_ask_bid']
quartile_3 = df[(df['spread_ask_bid'] > q2) & (df['spread_ask_bid'] <= q3)]['spread_ask_bid']
quartile_4 = df[df['spread_ask_bid'] > q3]['spread_ask_bid']


mean_q1 = quartile_1.mean()
mean_q2 = quartile_2.mean()
mean_q3 = quartile_3.mean()
mean_q4 = quartile_4.mean()


print("Moyenne Q1 (0-25%) :", mean_q1)
print("Moyenne Q2 (25-50%) :", mean_q2)
print("Moyenne Q3 (50-75%) :", mean_q3)
print("Moyenne Q4 (75-100%) :", mean_q4)

Moyenne Q1 (0-25%) : 0.0199788162686928
Moyenne Q2 (25-50%) : 0.036200714663151855
Moyenne Q3 (50-75%) : 0.05071257568725334
Moyenne Q4 (75-100%) : 0.10060265668959385


In [18]:
# import pandas as pd
# import plotly.graph_objs as go

# percentage_threshold = 0.002  # 0.005 MSTR, 

# data_time = 10  

# df = df_price
# df = df.sort_values('time').reset_index(drop=True)

# # Calcul des variations en pourcentage
# df['ask_pct_change'] = df['ask_price'].pct_change()
# df['bid_pct_change'] = df['bid_price'].pct_change()

# # Calcul de la différence de temps
# df['time_diff_ms'] = df['time'].diff()

# # Masque pour détecter les pics
# mask = (
#     ((df['ask_pct_change'].abs() > percentage_threshold) | (df['bid_pct_change'].abs() > percentage_threshold)) &
#     (df['time_diff_ms'] <= data_time)
# )

# # Filtrer les pics
# df_peaks = df[mask]

# # Inclure les lignes correspondantes pour le bid et l'ask au même timestamp
# df_spread_peaks = df[df['time'].isin(df_peaks['time'])].copy()

# # Convertir les timestamps en datetime pour l'axe x
# df['datetime'] = pd.to_datetime(df['time'], unit='ms')
# df_spread_peaks['datetime'] = pd.to_datetime(df_spread_peaks['time'], unit='ms')

# # Traces
# fig = go.Figure()

# # Tracer les prix ask
# fig.add_trace(go.Scatter(
#     x=df['datetime'],
#     y=df['ask_price'],
#     mode='lines',
#     line_shape='hv',
#     name='ask_price',
#     line=dict(color='blue', width=1)
# ))

# # Tracer les prix bid
# fig.add_trace(go.Scatter(
#     x=df['datetime'],
#     y=df['bid_price'],
#     mode='lines',
#     line_shape='hv',
#     name='bid_price',
#     line=dict(color='orange', width=1)
# ))

# # Marqueurs pour les pics (ask et bid au même timestamp)
# fig.add_trace(go.Scatter(
#     x=df_spread_peaks['datetime'],
#     y=df_spread_peaks['ask_price'],
#     mode='markers',
#     name='Ask Peaks',
#     marker=dict(color='red', size=6),
#     text=(df_spread_peaks['ask_pct_change'] * 100).round(3).astype(str) + '%',
#     hoverinfo='text+x+y'
# ))

# fig.add_trace(go.Scatter(
#     x=df_spread_peaks['datetime'],
#     y=df_spread_peaks['bid_price'],
#     mode='markers',
#     name='Bid Peaks',
#     marker=dict(color='green', size=6),
#     text=(df_spread_peaks['bid_pct_change'] * 100).round(3).astype(str) + '%',
#     hoverinfo='text+x+y'
# ))

# # Layout
# fig.update_layout(
#     title="Pics de prix avec spread",
#     xaxis_title="Temps",
#     yaxis_title="Prix",
#     hovermode='x unified',
#     template='plotly_white',
#     height=600
# )

# fig.show()